# Issue

Tunning this notebook, I got the following exception in Keras model evaluation method:
OutOfRangeError: 7 root error(s) found.
  (0) Out of range: {{function_node __inference_test_function_9112}} End of sequence
	 [[{{node cond_9/else/_84/cond_9/IteratorGetNext}}]]
	 [[strided_slice_33/_322]]
  (1) Out of range: {{function_node __inference_test_function_9112}} End of sequence
	 [[{{node cond_9/else/_84/cond_9/IteratorGetNext}}]]
	 [[tpu_compile_succeeded_assert/_5221419028491475094/_5/_351]]
  (2) Out of range: {{function_node __inference_test_function_9112}} End of sequence
	 [[{{node cond_9/else/_84/cond_9/IteratorGetNext}}]]
	 [[TPUReplicate/_compile/_12011285436106554079/_4/_300]]
  (3) Out of range: {{function_node __inference_test_function_9112}} End of sequence
	 [[{{node cond_9/else/_84/cond_9/IteratorGetNext}}]]
	 [[ConstantFoldingCtrl/cond_2/switch_pred/_20_1/_83]]
  (4) Out of range: {{function_node __inference_test_function_9112}} End of sequence
	 [[{{node cond_9/else/_84/cond_9/IteratorGetNext}}]]
  (5) Out of range: {{function_node __inference_test_function_9112}} End of sequence
	 [[{{node cond_9/else/_84/cond_9/IteratorGetNext}}]]
	 [[Shape_23/_220]]
  (6) Out of range: {{function_node __inference_test_function_9112}} End of sequence
	 [[{{node cond_9/else/_84/cond_9/IteratorGetNext}}]]
	 [[GroupCrossDeviceControlEdges_1/Identity_7/_395]]
0 successful operations.
2 derived errors ignored.

This notebook is a subset of my defaut [working notebook](https://www.kaggle.com/yanngarcia/human-protein-atlas) which is alredy shared. I removed all the useless code from Machine Learning and all my blabla ;)

I would like some help to understand what's going wrong.

Thanks a lot

**NOTE: Please, feel free to correct and enhance this notebook ;)**

Loading modules...

In [ ]:
from __future__ import division # Import floating-point division (1/4=0.25) instead of Euclidian division (1/4=0)

# 1. Prepare Problem

# a) Load libraries
import os, warnings, argparse, io, operator, requests, math, random, tempfile
import re # Regular expressions support
from datetime import datetime # Date & Time support

import numpy as np # Linear algebra
import matplotlib.pyplot as plt # Data visualization
import seaborn as sns # Enhanced data visualization
#import seaborn_image as isns # Enhanced image data visualization
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv)

from pandas_profiling import ProfileReport

import sklearn
from sklearn import model_selection
from sklearn import linear_model # Regression
from sklearn import discriminant_analysis
from sklearn import neighbors # Clustering
from sklearn import naive_bayes
from sklearn import tree # Decisional tree learning
from sklearn import svm # Support Vector Machines
from sklearn import ensemble # Support RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor

import xgboost as xgb # Gradient Boosted Decision Trees algorithm

import lightgbm as lgb # Light Gradient Boost model

from sklearn.base import is_classifier, is_regressor # To check if the model is for regression or classification (do not work for Keras)

from sklearn.impute import SimpleImputer 

from sklearn.preprocessing import LabelEncoder # Labelling categorical feature from 0 to N_class - 1('object' type)
from sklearn.preprocessing import LabelBinarizer # Binary labelling of categorical feature
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.preprocessing import StandardScaler # Data normalization
from sklearn.preprocessing import MinMaxScaler # Data normalization
from sklearn.preprocessing import MaxAbsScaler # Data normalization

from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import mutual_info_regression, mutual_info_classif # To build Mutual Information plots

from sklearn.inspection import permutation_importance

import pickle # Use to save and load models

import eli5
from eli5.sklearn import PermutationImportance

# Neural Network
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

from PIL import Image

Description of the different projects of my playground

In [ ]:

# Human Protein Atlas - Single Cell Classification (https://www.kaggle.com/c/hpa-single-cell-image-classification)
# Require ExecutionFlags.USE_CNN_NEURAL_NETWORK_FLAG
ML_NAME = 'HumanProteinAtlas' # https://www.kaggle.com/c/hpa-single-cell-image-classification
DATABASE_NAME = 'hpa-single-cell-image-classification'
COLUMNS_LABEL = ['image_id', 'class']
IMAGES_SAMPLES_NUM = 2048 # Total number of images to be used for all datasets. None: Use all images
IMAGE_NUM_PIXELS = 512 # Image size in pixels
IMAGE_SIZE = [IMAGE_NUM_PIXELS, IMAGE_NUM_PIXELS] # (Heigh,Width) image size in pixels
IMAGE_CLASSES = [ # Classes name
                    'Nucleoplasm', 
                    'Nuclear membrane', 
                    'Nucleoli', 
                    'Nucleoli fibrillar center', 
                    'Nuclear speckles', 
                    'Nuclear bodies', 
                    'Endoplasmic reticulum', 
                    'Golgi apparatus', 
                    'Intermediate filaments', 
                    'Actin filaments', 
                    'Microtubules', 
                    'Mitotic spindle', 
                    'Centrosome', 
                    'Plasma membrane', 
                    'Mitochondria', 
                    'Aggresome', 
                    'Cytosol', 
                    'Vesicles and punctate cytosolic patterns', 
                    'Negative'
                ]
TARGET_COLUMNS = None
OUTPUT_IS_REGRESSION = False
DATE_TIME_COLUMNS = None
DATE_TIME_COLUMNS = None
EXCLUDE_FROM_OULIERS = None
NON_TRANSFORMABLE_COLUMNS = None
# Channels:
# RED: Microtubule channels
# GREEN: Protein of interest
# BLUE: Nuclei channels
# YELLOW: Endoplasmic reticulum

Pre and Post processings...

In [ ]:
def kaggle_pre_load_images_datasets(p_train_url: str, 
                                    p_labels: list = None,
                                    p_global_path: str = None,
                                    p_validation_url: str = None,
                                    p_test_url: str = None,
                                    p_train_size: float = 0.9
                                    ) -> list:
    """
    This function is called by kaggle_load_images_datasets() just before to start processing
    The Images database shall be organized as follow:
    1) In Tfrecord format
        <current_folder>/tfrecords-jpeg-<n>x<n>/train
        <current_folder>/tfrecords-jpeg-<n>x<n>/val
        <current_folder>/tfrecords-jpeg-<n>x<n>/test
    2) Otherwise:
        <current_folder>/train_labels.csv
        <current_folder>/train
        [<current_folder>/train], could not exist
        <current_folder>/test
    :parameters p_labels: The label of the columns to be used. Default: None
    :parameter p_train_size: Size ratio of Training dataset vs. Validation dataset
    """
    print('----------------------------- kaggle_pre_load_images_datasets -----------------------------')
    train_df = None
    validation_df = None
    test_df = None
    
    if not p_train_url is None: # Case of separated Images and labels
        print('kaggle_pre_load_images_datasets: Processing %s' % ML_NAME)
        path = p_global_path #os.path.join(p_global_path, p_train_url) # p_train_url = DATABASE_NAME
        def _convert_multi_labels(p_labels: list, p_length: int, p_splitter: str = '|') -> list:
            labels = []
            for label in p_labels:
                l = np.zeros(p_length).astype(dtype = np.uint8)
                for item in label:
                    if(item != p_splitter):
                        l[int(item)] = int(1)
                labels.append(list(l))
            return labels
            # End of function _convert_multi_labels

        # Add a 'full path' feature with 4 color channels full path
        p = os.path.join(path, 'train.csv')
        train_df = pd.read_csv(p)
        # Set labels
        if not p_labels is None:
            train_df.columns = p_labels
        # Prepare Training, Validation and Test datasets
        print('----------------------------- training dataset')
        # Build full path images
        p = os.path.join(path, 'train')
        train_df['image_path'] = train_df['image_id'].apply(lambda x: p + '/%s' % x)
        train_df.drop(['image_id'], inplace = True, axis = 1)
        # Split labels into classes as descibed by IMAGE_CLASSES
        l = _convert_multi_labels(train_df['class'], p_length = len(IMAGE_CLASSES))
        classes = pd.DataFrame(l, columns = IMAGE_CLASSES)
        train_df.drop(['class'], inplace = True, axis = 1)
        train_df = pd.concat([train_df, classes], axis = 1)
        # Read the test files and build the files list
        print('----------------------------- kaggle_pre_load_images_datasets: test dataset')
        if not p_test_url is None:
            p = os.path.join(path, 'test')
            test_files = tf.io.gfile.glob(os.path.join(p, '*'))
            test_files = [(test_files[i].split('_')[0]) for i in range(0, len(test_files), 4)]
            # Put them in dataset
            test_df = pd.DataFrame(test_files, columns = ['image_path'])
        else:
            raise Exception('kaggle_pre_load_images_datasets', 'Test url not provided')
    
        if not p_validation_url is None:
            p = os.path.join(path, p_validation_url)
            if  ML_NAME == 'BMS-MolecularTranslation':
                # Add a 'full path' feature
                p = os.path.join(path, 'validation_labels.csv')
                validation_df = pd.read_csv(p)
                # Set labels
                if not p_labels is None:
                    validation_df.columns = p_labels
                validation_df['full_path'] = validation_df['image_id'].apply(lambda x: p + '/%c/%c/%c/%s.png' % (x[0], x[1], x[2], x))
            else:
                raise Exception('kaggle_pre_load_images_datasets', 'To be implemented')
        else: # Need to extract randomly Validation datatset from Training dataset
            train_df, validation_df = model_selection.train_test_split(train_df, test_size = p_train_size)
            # reindex after split
            train_df.reset_index(inplace = True)
            validation_df.reset_index(inplace = True)

        #print('----------------------------- kaggle_pre_load_images_datasets: training dataset')
        #print(train_df.head())
        #print('----------------------------- kaggle_pre_load_images_datasets: validation dataset')
        #print(validation_df.head())
        #print('----------------------------- kaggle_pre_load_images_datasets: test dataset')
        #print(test_df.head())
    else:
        pass

    print('kaggle_pre_load_images_datasets: Done')
    return train_df, validation_df, test_df

Loading the Datasets/Images

In [ ]:
# b.2) Set some defaults
def kaggle_set_mp_default() -> None:
    """
    Some default setting for Matplotlib plots
    """
    warnings.filterwarnings("ignore") # to clean up output cells
    pd.set_option('precision', 3)
    # Set Matplotlib defaults
    plt.rc('figure', autolayout=True)
    plt.rc('axes', labelweight='bold', labelsize='large', titleweight='bold', titlesize=18, titlepad=10)
    plt.rc('image', cmap='magma')
    # End of function set_mp_default

# Fix random values for reproductibility
SEED_HARCODED_VALUE = 42

def kaggle_set_seed(p_seed: int = SEED_HARCODED_VALUE) -> None:
    """
    Random reproducability
    :parameter p_seed: Set the seed value for random functions reproductibility
    """
    seed = p_seed
    if seed is None:
        seed = random.randint(0, 10000) 
    np.random.seed(seed)
    sklearn.utils.check_random_state(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    # End of function set_seed

def kaggle_modules_versions() -> None:
    """
    Print the different modules version
    """
    print('----------------------------- modules_versions -----------------------------')
    print("Numpy version: " + np.__version__)
    print('seaborn: %s' % sns.__version__)
    print("Pandas version: " + pd.__version__)
    print("Sklearn version: " + sklearn.__version__)
    print("Tensorflow version: " + tf.__version__)
    print('modules_versions: Done')
    # End of function modules_versions

In case of working with Neural Networks, set some additional defaults and doing a TPU detection are required.

In [ ]:
# b.4) Set additional defaults for Neural Networks
DL_BATCH_SIZE = 32 # Default batch size for DL models 
DL_EPOCH_NUM = 32 # Default epoch number for DL models 
DL_LEARNING_RATE = 0.002 # Default learning rate for DL models 
DL_DROP_RATE = 0.2 # Default drop rate for DL models 
DL_INPUT_SHAPE = None # Default input shape size for DL models, will be used for cross_validation (see kaggle_check_models)

# Doing TPU detection
def kaggle_tpu_detection():
    """
    This function provides a TPU detection. If TPU is not supported, a default strategy is returned.
    Note: This method also setup the global path to access datasets when TPU (from Kaggle environment) is detected.
    :return: The appropriate distribution strategy and the global path to access datasets (None if no TPU support)
    """
    print('----------------------------- kaggle_tpu_detection -----------------------------')
    global_path = None
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
        print('kaggle_tpu_detection: Running on TPU ', tpu.master())
    except ValueError:
        tpu = None
    if tpu:
        print("kaggle_tpu_detection: List of devices: ", tf.config.list_logical_devices('TPU'))
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        if strategy.num_replicas_in_sync > 1:
            # Update paths accordingly
            from kaggle_datasets import KaggleDatasets
            global_path = KaggleDatasets().get_gcs_path(DATABASE_NAME)
    else:
        strategy = tf.distribute.get_strategy() 
    print('kaggle_tpu_detection: replica=%s' % str(strategy.num_replicas_in_sync))
    print('kaggle_tpu_detection: global_path: ', global_path)
    
    print('kaggle_tpu_detection Done')
    return strategy, global_path
    # End of function kaggle_tpu_detection

# Tensorflow specific helper functions
def kaggle_bytes_to_tfrecord(p_value):
    """
    Returns a bytes_list from a string/byte
    """
    if isinstance(p_value, type(tf.constant(0))):
        p_value = p_value.numpy()
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [p_value]))
    # End of function kaggle_bytes_to_tfrecord

def kaggle_floats_to_tfrecord(p_value):
    """
    Returns a float_list from a float/double
    """
    return tf.train.Feature(float_list = tf.train.FloatList(value = [p_value]))
    # End of function kaggle_floats_to_tfrecord

def kaggle_ints_to_tfrecord(p_value):
    """
    Returns an int64_list from a bool/enum/int/uint
    """
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [p_value]))
    # End of function kaggle_ints_to_tfrecord

def kaggle_dataset_size(p_filenames: list) -> int:
    """
    The number of data items in the dataset is written in the name of the .tfrec files (i.g. features00-230.tfrec = 230 data items in the file features00-230.tfrec)
    """
    print('----------------------------- kaggle_dataset_size -----------------------------')
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in p_filenames]
    print('kaggle_dataset_size: n=', n)
    print('kaggle_dataset_size: np.sum(n)=', np.sum(n))
    return np.sum(n)
    # End of function kaggle_dataset_size

def kaggle_decode_png_image(p_image_data, p_channels: int = 3):
    """
    This function decodes the PNG image and applies required Tensorflow adjustment
    Note: The image was stored in a Tfrecord dataset
    :parameter p_image_data: 
    :parameter p_channels: 
    :return: The decoded image
    """
    image = tf.image.decode_png(p_image_data, channels = p_channels)
    # Convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0
    # Explicit size needed for TPU
    image = tf.reshape(image, [*IMAGE_SIZE, p_channels])
    return image
    # End of function kaggle_decode_png_image

def kaggle_decode_jpeg_image(p_image_data, p_channels: int = 3):
    """
    This function decodes the JPEG image and applies required Tensorflow adjustment
    Note: The image was stored in a Tfrecord dataset
    :parameter p_image_data: 
    :parameter p_channels: 
    :return: The decoded image
    """
    image = tf.image.decode_jpeg(p_image_data, channels = p_channels)
    # Convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0
    # Explicit size needed for TPU
    image = tf.reshape(image, [*IMAGE_SIZE, p_channels])
    return image
    # End of function kaggle_decode_jpeg_image

# Tensorflow Protocol buffer for unlabeled images stroed into a Tfrecord dataset.
# Labels are replaced by identifiers
unlabeled_image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
    'id': tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
}
def kaggle_read_unlabeled_tfrecord(p_data):
    data = tf.io.parse_single_example(p_data, unlabeled_image_feature_description)
    image = kaggle_decode_jpeg_image(data['image'])
    idnum = data['id']
    return image, idnum # returns a dataset of image(s)
    # End of function kaggle_read_unlabeled_tfrecord

# Tensorflow Protocol buffer for labeled images stroed into a Tfrecord dataset
labeled_image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
    'class': tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
}
def kaggle_read_labeled_tfrecord(p_data):
    data = tf.io.parse_single_example(p_data, labeled_image_feature_description)
    image = kaggle_decode_jpeg_image(data['image'])
    label = tf.cast(data['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs
    # End of function kaggle_read_labeled_tfrecord

Loads images dataset

In [ ]:
# c.2) Load 'Images' dataset
def kaggle_load_images_datasets(p_train_url: str, 
                                p_labels: list = None,
                                p_global_path: str = None,
                                p_validation_url: str = None,
                                p_test_url: str = None, 
                                p_train_path: str = None, 
                                p_validation_path: str = None,
                                p_test_path: str = None,
                                p_train_size: float = 0.9,
                                p_ordered: bool = False,
                                p_seed: int = SEED_HARCODED_VALUE
                                ) -> list:
    """
    This function loads Tensorflow Tfrecord datasets.
    
    :parameters p_labels: The label of the columns to be used. Default: None
    
    
    :parameter p_train_path: Path of the Training 'Tfrec' folder  
    :parameter p_validation_path: Path of the Validation 'Tfrec' folder
    :parameter p_test_path: Path of the Test 'Tfrec' folder
    :parameter p_train_size: Size ratio of Training dataset vs. Validation dataset
    :parameter p_seed: The seed value for reproductibility
    :return: Four datasets: The Training, Test and Validation datasets. The Test dataset does no contain the outputs, it acts as unseen data for the model. This is the fourth dataset returned
    """
    print('----------------------------- kaggle_load_images_datasets -----------------------------')

    train_df, validation_df, test_df = kaggle_pre_load_images_datasets(p_train_url = p_train_url, 
                                                                       p_labels = p_labels, 
                                                                       p_global_path = p_global_path,
                                                                       p_validation_url = p_validation_url, 
                                                                       p_test_url = p_test_url,
                                                                       p_train_size = p_train_size
                                                                       )
    #print('----------------------------- kaggle_load_images_datasets: training dataset')
    #print(train_df.head())
    #print('----------------------------- kaggle_load_images_datasets: validation dataset')
    #print(validation_df.head())
    #print('----------------------------- kaggle_load_images_datasets: test dataset')
    #print(test_df.head())

    train_df_num_images = None
    validation_df_num_images = None
    test_df_num_images = None
    if not train_df is None:
        print('kaggle_load_images_datasets: Processing compete %s:' % ML_NAME)
        if ML_NAME == 'HumanProteinAtlas':
            def _rebuild_image_from_channels(p_image_path: str, p_label: list = None, p_seed: int = SEED_HARCODED_VALUE) -> list:
                """
                This function convert image channels into one RGB image
                """
                red = tf.io.read_file(p_image_path + "_red.png")
                blue = tf.io.read_file(p_image_path + "_blue.png")
                green = tf.io.read_file(p_image_path + "_green.png")
                yellow = tf.io.read_file(p_image_path + "_yellow.png")

                red = tf.io.decode_png(red, channels = 1) # Grayscale image
                blue = tf.io.decode_png(blue, channels = 1) # Grayscale image
                green = tf.io.decode_png(green, channels = 1) # Grayscale image
                yellow = tf.io.decode_png(yellow, channels = 1) # Grayscale image
                
                red = tf.math.maximum(red, yellow)
                blue = tf.math.maximum(blue, yellow)

                # Convert image to floats in [0, 1] range
                red = tf.cast(red, tf.float32) / 255.0
                blue = tf.cast(blue, tf.float32) / 255.0
                green = tf.cast(green, tf.float32) / 255.0
                
                # Explicit size needed for TPU
                red = tf.image.resize(red, [*IMAGE_SIZE])
                blue = tf.image.resize(blue, [*IMAGE_SIZE])
                green = tf.image.resize(green, [*IMAGE_SIZE])

                red = tf.squeeze(red)
                blue = tf.squeeze(blue)
                green = tf.squeeze(green)
                
                # Build RGB image, channels last: shape is (width, high, channel)
                # In this case, shape is (1024, 1024, 3):
                #     1024 entries of (1024 lines per 3 columns)
                #         First entry, first line:   [ R[0,0], G[0,0], B[0,0] ]
                #         First entry, second line:  [ R[0,1], G[0,1], B[0,1] ]
                #         ...
                #         Second entry, first line:  [ R[1,0], G[1,0], B[1,0] ]
                #         ....
                # Stack structure is: each entry of the stack is the tuple (R, G, B) of  
                # Axis value: 2 for channels last, -1 for channel first
                # From stack, to extract one channel (R, G or B): stack[:, :, n], n = 0 for R, 1, for G and 2 for B
                image = tf.stack([red, green, blue], axis = 2) # RGB channels last

                return image, p_label
                # End of _rebuild_image_from_channels

            # Sample elements from Tests and Validation dataset to reduce execution time
            if not IMAGES_SAMPLES_NUM is None: # Use a subset of images from the training Dataset
                # Shuffle row from the Training dataset
                train_df = train_df.sample(IMAGES_SAMPLES_NUM, random_state = p_seed)
                train_df.reset_index(inplace = True);
                # Shuffle row from the Validation dataset
                validation_df = validation_df.sample(int((1 - p_train_size) * IMAGES_SAMPLES_NUM // p_train_size), random_state = p_seed)
                validation_df.reset_index(inplace = True);
            else: # Use all images from the training Dataset
                # Nothing to do
                pass
            # Set sizes
            train_df_num_images = train_df.shape[0]
            validation_df_num_images = validation_df.shape[0]
            test_df_num_images = test_df.shape[0]
            # Build Tensoflow dataset for Training
            image_path = train_df['image_path'] # List of image paths as pd.Series
            labels = train_df[IMAGE_CLASSES] # List of labels as pd.DataFrame
            print('Training image_path shape ==> ', image_path.shape)
            print('Training labels shape ==> ', labels.shape)
            # Conver Series/DataFrame into np.darray before to convert them into tensors
            image_path = np.array(image_path.values.tolist())
            #print(type(image_path))
            #print(image_path)
            labels = np.array(labels.values.tolist())
            #print(type(labels))
            #print(labels)
            # Testing _rebuild_image_from_channels function
            #i, t = _rebuild_image_from_channels(image_path[0], labels[0])
            #print(i.shape)
            #print(i)
            #print(t)
            #raise Exception('Stop')
            # Convert them into tensors
            train_df = tf.data.Dataset.from_tensor_slices((image_path, labels))
            # Shuffle the training dataset
#            train_df = train_df.shuffle(len(image_path) + 1)  # buffer_size >= dataset length
            train_df = train_df.map(_rebuild_image_from_channels, num_parallel_calls = tf.data.AUTOTUNE)
            # Inspect Training dataset
            for image, label in train_df.take(3):
                print('kaggle_load_images_datasets: Training data label inspection:', label.numpy())
                print(image.numpy().shape, len(image.numpy()), label.numpy().shape)
            # Build Tensoflow dataset for Validation
            image_path = validation_df['image_path'] # List of image paths as pd.Series
            labels = validation_df[IMAGE_CLASSES] # List of labels as pd.DataFrame
            print('Validation image_path shape ==> ', image_path.shape)
            print('Validation labels shape == ', labels.shape)
            # Conver Series/DataFrame into np.darray before to convert them into tensors
            image_path = np.array(image_path.values.tolist())
            #print(type(image_path))
            #print(image_path)
            labels = np.array(labels.values.tolist())
            #print(type(labels))
            #print(labels)
            # Convert them into tensors
            validation_df = tf.data.Dataset.from_tensor_slices((image_path, labels))
            # Shuffle the validation dataset
#            validation_df = validation_df.shuffle(len(image_path) + 1)  # buffer_size >= dataset length
            validation_df = validation_df.map(_rebuild_image_from_channels, num_parallel_calls = tf.data.AUTOTUNE)
            # Inspect Validation dataset
            for image, label in validation_df.take(3):
                print('kaggle_load_images_datasets: Validation data label inspection:', label.numpy())
                print(image.numpy().shape, len(image.numpy()), label.numpy().shape)
            # Build Tensoflow dataset for Validation
            image_path = test_df['image_path'] # List of image paths
            # Conver Series/DataFrame into np.darray before to convert them into tensors
            image_path = np.array(image_path.values.tolist())
            #print('type(image_path) ==> ', type(image_path))
            #print(image_path)
            test_df = tf.data.Dataset.from_tensor_slices(image_path)
            # Shuffle the test dataset
#            test_df = test_df.shuffle(len(image_path) + 1)  # buffer_size >= dataset length
            test_df = test_df.map(_rebuild_image_from_channels, num_parallel_calls = tf.data.AUTOTUNE)
            # Inspect Validation dataset
            for image, label in validation_df.take(3):
                print('kaggle_load_images_datasets: Test data inspection:')
                print(image.numpy().shape, len(image.numpy()))
        #elif ML_NAME == 'BMS-MolecularTranslation':
        #    pass
        else:
            raise Exception('kaggle_load_images_datasets', 'Unsupported ML_NAME: %s' % ML_NAME)
    elif ML_NAME == 'FlowerClassification':
        print('kaggle_load_images_datasets: Processing compete %s:' % ML_NAME)
        # Load file names for Train, Validation and Test images folders
        train_files = tf.io.gfile.glob(os.path.join(p_global_path, p_train_path))
        # Sample elements from Tests and Validation dataset to reduce execution time
        # FIXME How to sample Tensorflow datasets?
        #if not IMAGES_SAMPLES_NUM is None: # Use a subset of images from the training Dataset
        #    # Shuffle row from the Training dataset
        #    train_files = random.sample(train_files, IMAGES_SAMPLES_NUM)
        #    train_files.reset_index(inplace = True);
        #else: # Use all images from the training Dataset
        #    # Nothing to do
        #    pass
        # Disabling order increases speed
        ignore_order = tf.data.Options()
        if not p_ordered:
            ignore_order.experimental_deterministic = False # disable order, increase speed
        # Build the dataset with the images
        train_df = tf.data.TFRecordDataset(train_files, num_parallel_reads = tf.data.experimental.AUTOTUNE)
        # To use data as soon as it streams in
        train_df = train_df.with_options(ignore_order)
        # Decode tfrecord images into (jpeg, label)
        train_df = train_df.map(kaggle_read_labeled_tfrecord, num_parallel_calls = tf.data.experimental.AUTOTUNE)
        print('kaggle_load_images_datasets: Training data shapes:', train_df.cardinality().numpy())
        # Inspect Training dataset
        for image, label in train_df.take(3):
            print('kaggle_load_images_datasets: Training data label inspection:', label.numpy())
            print(image.numpy().shape, len(image.numpy()), label.numpy().shape)
        train_df_num_images = kaggle_dataset_size(train_files)
        if not p_validation_path is None:
            validation_files = tf.io.gfile.glob(os.path.join(p_global_path, p_validation_path))
            # Sample elements from Tests and Validation dataset to reduce execution time
            # FIXME How to sample Tensorflow datasets?
            #if not IMAGES_SAMPLES_NUM is None: # Use a subset of images from the training Dataset
            #    # Shuffle row from the Validation dataset
            #    validation_df = random.sample(validation_df, int((1 - p_train_size) * IMAGES_SAMPLES_NUM // p_train_size))
            #    validation_df.reset_index(inplace = True);
            #else: # Use all images from the validation Dataset
            #    # Nothing to do
            #    pass
            # Build the dataset with the images
            validation_df = tf.data.TFRecordDataset(validation_files, num_parallel_reads = tf.data.experimental.AUTOTUNE)
            # To use data as soon as it streams in
            validation_df = validation_df.with_options(ignore_order)
            # Decode tfrecord images into (jpeg, label)
            validation_df = validation_df.map(kaggle_read_labeled_tfrecord, num_parallel_calls = tf.data.experimental.AUTOTUNE)
            print('kaggle_load_images_datasets: Validation data shapes:', validation_df.cardinality().numpy())
            # Inspect Validation dataset
            for image, label in validation_df.take(3):
                print('kaggle_load_images_datasets: Validation data label inspection:', label.numpy())
                print(image.numpy().shape, len(image.numpy()), label.numpy().shape)
            validation_df_num_images = kaggle_dataset_size(validation_files)
        if not p_test_path is None:
            test_files = tf.io.gfile.glob(os.path.join(p_global_path, p_test_path))
            print('----------------------------- kaggle_load_images_datasets: test_files')
            # Build the dataset with the images
            test_df = tf.data.TFRecordDataset(test_files, num_parallel_reads = tf.data.experimental.AUTOTUNE)
            # To use data as soon as it streams in
            test_df = test_df.with_options(ignore_order)
            # Decode tfrecord images into (jpeg, label)
            test_df = test_df.map(kaggle_read_unlabeled_tfrecord, num_parallel_calls = tf.data.experimental.AUTOTUNE)
            # Inspect Test dataset
            for image in test_df.take(3):
                print('kaggle_load_images_datasets: Test data inspection:')
                print(image.numpy().shape, len(image.numpy()))
            print('kaggle_load_images_datasets: Test data shapes:', test_df.cardinality().numpy())
            test_df_num_images = kaggle_dataset_size(test_files)

    print('kaggle_load_images_datasets: Done: %s' % (p_train_url if not p_train_url is None else p_train_path))
    return train_df, validation_df, test_df, train_df_num_images, validation_df_num_images, test_df_num_images
    # End of function kaggle_load_images_datasets

Image visualization...

In [ ]:
# b.2) Images visualization
def kaggle_image_visualization(p_dataset, p_num_images: int = 10, p_prediction = None):
    """
    This method show a number of images from p_dataset
    :parameter p_dataset: The dataset containing the images to show. Note the dataset is usually already 'batch'
    :parameter p_num_images: The number of images to show
    :parameter p_prediction: 
    """
    print('----------------------------- kaggle_image_visualization -----------------------------')
    # Peek some data from the dataset
    it = iter(p_dataset.unbatch().batch(p_num_images))
    batch = next(it)
    kaggle_display_batch(batch, p_prediction)
    # End of function kaggle_image_visualization

def kaggle_batch_to_numpy(p_dataset):
    print('----------------------------- kaggle_batch_to_numpy -----------------------------')
    images, labels = p_dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case,
                                     # these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is
    # the case for test data)
    return numpy_images, numpy_labels
    # End of function kaggle_batch_to_numpy

def kaggle_title_from_label(p_label: str, p_correct_label: str):
    print('----------------------------- kaggle_title_from_label -----------------------------')
    if p_correct_label is None:
        return IMAGE_CLASSES[p_label], True
    correct = (p_label == p_correct_label)
    return "{} [{}{}{}]".format(IMAGE_CLASSES[p_label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                IMAGE_CLASSES[p_correct_label] if not correct else ''), correct
    # End of function kaggle_title_from_label

def kaggle_display_batch_image(p_image, p_title:str, subplot, red = False, titlesize = 16, p_cmap = 'viridis'):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(p_image, cmap = p_cmap)
    if len(p_title) > 0:
        plt.title(p_title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    # End of function kaggle_display_batch_image
    
def kaggle_display_batch(p_databatch, p_predictions = None):
    """
    """
    print('----------------------------- kaggle_display_batch -----------------------------')
    # Images
    images, labels = kaggle_batch_to_numpy(p_databatch)
    if labels is None: # Fill with None
        labels = [None for _ in enumerate(images)] 
    # Auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
    # Sizing and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows, cols, 1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE / cols * rows))
    else:
        plt.figure(figsize=(FIGSIZE / rows * cols,FIGSIZE))
    # Display
    for i, (image, label) in enumerate(zip(images[:rows * cols], labels[:rows * cols])):
        title = 'No label' if label is None or type(label) != 'int' else IMAGE_CLASSES[label]
        correct = True
        if not p_predictions is None:
            title, correct = kaggle_title_from_label(p_predictions[i], label)
        dynamic_titlesize = FIGSIZE * SPACING / max(rows, cols) * 40 + 3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = kaggle_display_batch_image(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    # Layout
    plt.tight_layout()
    if label is None and p_predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()
    # End of function kaggle_display_batch

def kaggle_display_image_and_component(p_image, p_title:str = None) -> None:
    """
    This function displays an image and its RGB components separatly
    :parameter p_image: The image to display (RGB format)
    :parameter p_title: The title of the display
    """
    # Extract RGB components
    pixels = img_to_array(p_image)
    red = pixels[:, :, 0]
    green = pixels[:, :, 1]
    blue = pixels[:, :, 2]
    # Sizing and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    rows = 1
    cols = 4
    subplot=(rows, cols, 1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE / cols * rows))
    else:
        plt.figure(figsize=(FIGSIZE / rows * cols,FIGSIZE))
    # Display image and its components
    images = (pixels, red, green, blue)
    titles = (p_title, 'red component', 'green component', 'blue component')
    for i, (image, title) in enumerate(zip(images, titles)):
        dynamic_titlesize = FIGSIZE * SPACING / max(rows, cols) * 40 + 3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = kaggle_display_batch_image(image, title, subplot, True, titlesize = dynamic_titlesize, p_cmap = 'viridis' if i == 0 else 'gray')    
    # Layout
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()
    # End of function kaggle_display_image_and_component

def kaggle_display_confusion_matrix(cmat, score, precision, recall) -> None:
    """
    """
    print('----------------------------- kaggle_display_confusion_matrix -----------------------------')
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(IMAGE_CLASSES)))
    ax.set_xticklabels(IMAGE_CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(IMAGE_CLASSES)))
    ax.set_yticklabels(IMAGE_CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()
    print('kaggle_display_confusion_matrix: Done')
    # End of function kaggle_display_confusion_matrix

def kaggle_display_learning_curves(training, validation, title, subplot) -> None:
    print('----------------------------- kaggle_display_learning_curves -----------------------------')
    if subplot % 10 == 1: # set up the subplots on the first call
        plt.subplots(figsize = (10,10), facecolor = '#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    plt.show()
    print('kaggle_display_learning_curves: Done')
    # End of function kaggle_display_learning_curves

Callbacks to create DL models and functions to visualize learning curves or learning rate curves

In [ ]:
# Helper functions for kaggle_dl_quick_and_dirty()
def kaggle_build_pretrained_model(
                                  p_pretrained_layers:list, 
                                  p_image_size: list,
                                  p_channels: int = 3
                                  ) -> tf.keras.Sequential:
    """
    This function builds the pretrained models stacks for the Neural Network
    :parameter p_pretrained_layers: The pretrained models to be used
    :parameter p_image_size: The image size
    :parameter p_channels: The number of channels (e.g. 1 for grayscale, 3 for RGB)
    :return: The pretrained models stacks to build the final Neural Network
    """
    model = tf.keras.Sequential();
    for p in p_pretrained_layers:
        if p == 'DenseNet201': 
            pretrained_model = tf.keras.applications.DenseNet201(
                weights = 'imagenet',
                include_top = False,
                pooling = 'max',
                input_shape = [*p_image_size, p_channels]
            )
            pretrained_model.trainable = False
            model.add(pretrained_model)
            model.add(GlobalAveragePooling2D())
        elif p == 'InceptionV3': 
            # See https://github.com/fchollet/deep-learning-models/blob/ccd0eb24996b4cbff4231b90cd44b057c0b20f14/inception_v3.py
            pretrained_model = tf.keras.applications.InceptionV3(
                weights = 'imagenet',
                include_top = False,
                pooling = 'max',
                input_shape = [*p_image_size, p_channels]
            )
            pretrained_model.trainable = False
            model.add(pretrained_model)
        elif p == 'MobileNetV2':
            pretrained_model = tf.keras.applications.MobileNetV2(
                weights = 'imagenet',
                include_top = False,
                pooling = 'max',
                input_shape = [*p_image_size, p_channels]
            )
            pretrained_model.trainable = False
            model.add(pretrained_model)
        elif p == 'ResNet50':
            # See https://github.com/fchollet/deep-learning-models/blob/ccd0eb24996b4cbff4231b90cd44b057c0b20f14/resnet50.py
            pretrained_model = tf.keras.applications.ResNet50(
                weights = 'imagenet',
                include_top = False ,
                input_shape = [*p_image_size, p_channels]
            )
            pretrained_model.trainable = False
            model.add(pretrained_model)
        elif p == 'VGG16': 
            # See https://github.com/fchollet/deep-learning-models/blob/master/vgg16.py
            pretrained_model = tf.keras.applications.VGG16(
                weights = 'imagenet',
                include_top = False ,
                input_shape = [*p_image_size, p_channels]
            )
            pretrained_model.trainable = False
            model.add(pretrained_model)
        elif p == 'VGG19':
            # See https://github.com/fchollet/deep-learning-models/blob/master/vgg19.py
            pretrained_model = tf.keras.applications.VGG19(
                weights = 'imagenet',
                include_top = False,
                pooling = 'max',
                input_shape = [*p_image_size, p_channels]
            )
            pretrained_model.trainable = False
            model.add(pretrained_model)
        else:
            raise Exception('kaggle_build_pretrained_model', 'Undefined pretrained model')
        # End of 'for' statement
    return model
    # End of function kaggle_build_pretrained_model

def kaggle_create_sequential_classifier_model(
                                              #p_strategy
                                              #p_input_shape: int = DL_INPUT_SHAPE,
                                              #p_drop_rate: float = DL_DROP_RATE,
                                              p_optimizer: str = 'adam', 
                                              p_loss: str = 'binary_crossentropy', 
                                              p_metrics: list = ['AUC', 'accuracy'],
                                              p_pretrained_layers:list = None,
                                              p_image_size: list = None,
                                              p_class_num: int = 1
                                             ) -> tf.keras.Sequential:
    """
    Build a Neural network model for classification
    """
    print('----------------------------- kaggle_create_sequential_classifier_model -----------------------------')
    global DL_INPUT_SHAPE, DL_DROP_RATE

    print('kaggle_create_sequential_classifier_model: p_class_num=%d' % p_class_num)
    model = None
    if p_pretrained_layers is None:
        model = tf.keras.Sequential([
                                    tf.keras.layers.BatchNormalization(input_shape = DL_INPUT_SHAPE),
                                    tf.keras.layers.Dense(32, activation = 'relu'),
                                    tf.keras.layers.Dropout(rate = DL_DROP_RATE),
                                    tf.keras.layers.BatchNormalization(),
                                    tf.keras.layers.Dense(64, activation = 'relu'),
                                    tf.keras.layers.BatchNormalization(),
                                    tf.keras.layers.Dropout(rate = DL_DROP_RATE),
                                    tf.keras.layers.Dense(p_class_num, activation = 'sigmoid'), # Binary classes
                                    ])
    else:
        model = kaggle_build_pretrained_model(p_pretrained_layers, p_image_size)
        model.add(tf.keras.layers.Dense(p_class_num, activation = 'softmax')) # Multi classes

    model.compile(optimizer=p_optimizer, loss = p_loss, metrics = p_metrics)
    # End of 'with' statement

    print('kaggle_create_sequential_classifier_model: Model summary:')
    model.summary()
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    return model
    # End of function kaggle_create_sequential_classifier_model

def kaggle_create_sequential_regressor_model(
                                             p_optimizer:str = 'adam', 
                                             p_loss:str = 'mae', 
                                             p_metrics:list = ['mae'],
                                             p_pretrained_layers: list = None,
                                             p_image_size: list = None,
                                             p_class_num: int = 1
                                            ) -> tf.keras.Sequential:
    """
    Build a Neural network model for regression
    """
    print('----------------------------- kaggle_create_sequential_regressor_model -----------------------------')
    global DL_INPUT_SHAPE, DL_DROP_RATE

    model = None
    if p_pretrained_layers is None:
        model = tf.keras.Sequential([
                                    tf.keras.layers.BatchNormalization(input_shape = DL_INPUT_SHAPE),
                                    tf.keras.layers.Dense(32, activation = 'relu'),
                                    tf.keras.layers.Dropout(rate = DL_DROP_RATE),
                                    tf.keras.layers.BatchNormalization(),
                                    tf.keras.layers.Dense(64, activation = 'relu'),
                                    tf.keras.layers.BatchNormalization(),
                                    tf.keras.layers.Dropout(rate = DL_DROP_RATE),
                                    tf.keras.layers.Dense(p_class_num, activation = 'relu'),
                                    ])
    else:
        model = kaggle_build_pretrained_model(p_pretrained_layers, p_image_size);
        model.add(tf.keras.layers.Dense(p_class_num, activation='relu'))

    model.compile(optimizer=p_optimizer, loss = p_loss, metrics = p_metrics)

    print('kaggle_create_sequential_regressor_model: Model summary:')
    model.summary()
    return model
    # End of function kaggle_create_sequential_regressor_model

def kaggle_build_learning_rate_schedule(epoch,
                                        start_lr = 0.0001, 
                                        min_lr = 0.0001, 
                                        max_lr = 0.0005,
                                        rampup_epochs = 5, 
                                        sustain_epochs = 0,
                                        exp_decay = 0.8
                                       ):
    """
    """
    def _exponential_lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        """
        """
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr = ((max_lr - start_lr) /
                  rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        # exponential decay towards min_lr
        else:
            lr = ((max_lr - min_lr) *
                  exp_decay**(epoch - rampup_epochs - sustain_epochs) +
                  min_lr)
        return lr
    l = _exponential_lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)
    return l
    # End of function kaggle_build_learning_rate_schedule

def kaggle_display_learning_rate_curve(p_lr):
    print('----------------------------- kaggle_display_learning_rate_curve -----------------------------')
    rng = [i for i in range(DL_EPOCH_NUM)]
    y = [p_lr(x) for x in rng]
    plt.plot(rng, y)
    plt.show()
    print("kaggle_display_learning_rate_curve: Done: Schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))
    # End of function kaggle_display_learning_rate_curve

Data augmentation... 

In [ ]:
def kaggle_image_augmentation(
                              p_dataset, 
                              p_strategy, 
                              p_augmentations: list = None, 
                              p_shuffle_buffer_size: int = 2048, 
                              p_seed: int = SEED_HARCODED_VALUE
                              ):
    """
    This function proceeds to data augmentation in order to enhance Training dataset by creating new images
    :parameter p_seed: The seed value for reproductibility
    Note: Using method dataset.prefetch(tf.data.experimental.AUTOTUNE), 
          this processing happens essentially for free on TPU. 
          Data pipeline code is executed on the "CPU" part of the TPU 
          while the TPU itself is computing gradients.
    """
    print('----------------------------- kaggle_image_augmentation -----------------------------')
    if not p_augmentations is None:
        for augmentation in p_augmentations:
            if augmentation == 'flip_right_left':
                dataset = p_dataset.map(kaggle_flip_left_right_data_augmentation, num_parallel_calls = tf.data.experimental.AUTOTUNE)
            elif augmentation == 'flip_up_down':
                dataset = p_dataset.map(kaggle_flip_up_down_data_augmentation, num_parallel_calls = tf.data.experimental.AUTOTUNE)
            elif augmentation == 'random_contrast':
                dataset = p_dataset.map(kaggle_random_contrast_data_augmentation, num_parallel_calls = tf.data.experimental.AUTOTUNE)
            elif augmentation == 'random_brightness':
                dataset = p_dataset.map(kaggle_random_brightness_data_augmentation, num_parallel_calls = tf.data.experimental.AUTOTUNE)
            else:
                raise Exception('kaggle_image_augmentation', 'Wrong parameter')
            # End of 'for' statement
        dataset = dataset.repeat() # the training dataset must repeat for several epochs
        dataset = dataset.shuffle(p_shuffle_buffer_size, seed = p_seed) # Shuffle images of the batch at each iteration
        dataset = dataset.batch(DL_BATCH_SIZE * p_strategy.num_replicas_in_sync) # Prepare batches for DL
    else:
        dataset = p_dataset.batch(DL_BATCH_SIZE * p_strategy.num_replicas_in_sync) # Prepare batches for DL
        
    if p_augmentations is None:
        dataset = dataset.cache()
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE) # The next batch will always be ready for processing for the next iteration
    print('kaggle_image_augmentation: type(dataset) => ', type(dataset))
    print('kaggle_image_augmentation: Done')
    return dataset
    # End of function kaggle_image_augmentation  

def kaggle_flip_left_right_data_augmentation(p_image, p_label):
    """
    """
    print('----------------------------- kaggle_flip_left_right_data_augmentation -----------------------------')
    image = tf.image.random_flip_left_right(p_image)
    print('kaggle_flip_left_right_data_augmentation: Done')
    return image, p_label
    # End of function kaggle_flip_left_right_data_augmentation

def kaggle_flip_up_down_data_augmentation(p_image, p_label):
    """
    """
    print('----------------------------- kaggle_flip_up_down_data_augmentation -----------------------------')
    image = tf.image.random_flip_left_right(p_image)
    print('kaggle_flip_up_down_data_augmentation: Done')
    return image, p_label
    # End of function kaggle_flip_up_down_data_augmentation

def kaggle_random_contrast_data_augmentation(p_image, p_label):
    """
    """
    print('----------------------------- kaggle_flip_left_right_data_augmentation -----------------------------')
    image = tf.image.random_contrast(p_image, lower = 0.3, upper = 1.2)
    print('kaggle_random_contrast_data_augmentation: Done')
    return image, p_label
    # End of function kaggle_random_contrast_data_augmentation

def kaggle_random_brightness_data_augmentation(p_image, p_label):
    """
    """
    print('----------------------------- kaggle_random_brightness_data_augmentation -----------------------------')
    image = tf.image.random_brightness(p_image, lower = 0.3, upper = 1.2)
    print('kaggle_random_brightness_data_augmentation: Done')
    return image, p_label
    # End of function kaggle_random_brightness_data_augmentation

In [ ]:
# b.2) Check models for CNN
def kaggle_check_dl_model(p_strategy, p_model, p_train_df, p_train_df_size, p_validation_df, p_validation_df_size):
    """
    """
    print('----------------------------- kaggle_check_dl_model -----------------------------')
    with p_strategy.scope():
        print('kaggle_check_dl_model: Setup LearningRateScheduler:')
        
        lr_callback = tf.keras.callbacks.LearningRateScheduler(kaggle_build_learning_rate_schedule, verbose = True)
        kaggle_display_learning_rate_curve(kaggle_build_learning_rate_schedule)
        temp_name = next(tempfile._get_candidate_names()) + '.h5'
        # Training the model
        print('kaggle_check_dl_model: Train the model:')
        print('kaggle_check_dl_model: steps_per_epoch = ', p_train_df_size // (DL_BATCH_SIZE * p_strategy.num_replicas_in_sync))
        history = p_model.fit(
            p_train_df,
            validation_data = p_validation_df,
            epochs = DL_EPOCH_NUM,
            steps_per_epoch = p_train_df_size // (DL_BATCH_SIZE * p_strategy.num_replicas_in_sync), # p_train_df_size is the initial size of the Train dataset.
                                                                                                    # After data augmentation, this size is much much higher
            callbacks = [lr_callback, ModelCheckpoint(filepath = temp_name, monitor = 'val_loss', save_best_only = True)],
            verbose = 2
        )
        # End of 'with' statement

    # Display leaning curve
    kaggle_display_learning_curves(
        history.history['loss'],
        history.history['val_loss'],
        'loss',
        211,
    )
    kaggle_display_learning_curves(
        history.history['sparse_categorical_accuracy'],
        history.history['val_sparse_categorical_accuracy'],
        'accuracy',
        212,
    )
    # Show scores
    # Split Validation images & labels
    print('kaggle_check_dl_model: Split images & labels for validation_df')
    validation_images_df = p_validation_df.map(lambda image, label: image)
    validation_labels_df = p_validation_df.map(lambda image, label: label).unbatch()
    # Convert labels into a Pandas dataframe
    #l = []
    #for i, e in validation_labels_df.enumerate():
    #    l[i] = int(e.numpy())
    #validation_labels_df = pd.DataFrame(l)
    #print('type(validation_labels_df) ==>', type(validation_labels_df))
    #print('Main: validation_labels_df.shape = ', len(validation_labels_df), ' / ', validation_df_size)
    #raise Exception('Stop')

    # Display confusion matrix
    cm_correct_labels = next(iter(validation_labels_df.batch(p_validation_df_size))).numpy() #np.darray
    print('kaggle_check_dl_model: cm_correct_labels ==>', cm_correct_labels)
    cm_probabilities = kaggle_prediction(p_model, validation_images_df)
    print('kaggle_check_dl_model: cm_probabilities ==>', cm_probabilities)
    y_predictions = np.argmax(cm_probabilities, axis = -1)
    print('kaggle_check_dl_model: y_predictions ==>', y_predictions)
    labels = range(len(IMAGE_CLASSES))
    cmat = confusion_matrix(
                            cm_correct_labels,
                            y_predictions,
                            labels = labels,
                            )
    cmat = (cmat.T / cmat.sum(axis=1)).T # normalize
    print('kaggle_check_dl_model: cmat = ', cmat)
    # Display scores
    score = f1_score(
        cm_correct_labels,
        y_predictions,
        labels=labels,
        average = 'macro',
    )
    print('kaggle_check_dl_model: Score = ', score)
    precision = precision_score(
                                cm_correct_labels,
                                y_predictions,
                                labels = labels,
                                average = 'macro',
    )
    print('kaggle_check_dl_model: Precision score = ', precision)
    recall = recall_score(
                          cm_correct_labels,
                          y_predictions,
                          labels = labels,
                          average='macro',
    )
    print('kaggle_check_dl_model: Recall score = ', recall)
    kaggle_display_confusion_matrix(cmat, score, precision, recall)

    # Retrieve the best weights for the model
    print('kaggle_check_dl_model: Retrieve the best weights for the model')
    model = tf.keras.models.load_model(temp_name)
    os.remove(temp_name)

    print('kaggle_check_dl_model: Done')
    return (score, precision, recall), history.history, model
    # End of function kaggle_check_dl_model

Main function

In [ ]:
def kaggle_main() -> None:
    global DL_INPUT_SHAPE
    
    # Set defaults
    kaggle_set_seed()
    kaggle_set_mp_default()
    
    # Current path
    print(os.path.abspath(os.getcwd()))
    # Kaggle current path and files
    #for dirname, _, filenames in os.walk('/kaggle/input'):
    #    for filename in filenames:
    #        print(os.path.join(dirname, filename))

    # Modules version
    kaggle_modules_versions()

    print('Main: Playgrounf name: ', ML_NAME)

    strategy, global_path = kaggle_tpu_detection()
    if global_path is None: # Force global_path to local path when datasets not copied to gs://kds-xxx
        global_path = os.path.join(os.path.abspath(os.getcwd()), '../input')
        global_path = os.path.join(global_path, DATABASE_NAME)

    # Images and lables are separated
    train_df, validation_df, test_df, train_df_size, validation_df_size, test_df_size = kaggle_load_images_datasets(p_train_url = DATABASE_NAME,
                                                                                                                    p_labels = COLUMNS_LABEL, 
                                                                                                                    p_global_path = global_path,
                                                                                                                    p_test_url = DATABASE_NAME
                                                                                                                    )

    print('type(train_df) ==> ', type(train_df))
    print('Main: train_df_size', train_df_size)
    print('type(validation_df) ==> ', type(validation_df))
    print('Main: validation_df_size', validation_df_size)
    print('type(test_df_size) ==> ', type(train_df))
    print('Main: test_df_size', test_df_size)
    # End of datasets loading operation

#    raise Exception('Stop')
    # 1. Data augmentation and batches setup
    augmentations_list = ['flip_right_left']
    train_df = kaggle_image_augmentation(train_df, strategy, augmentations_list)
    train_df_size *= len(augmentations_list) 
    # Inspect Validation dataset
    for image_batch, label_batch in train_df.take(1):
        pass
    print('Main: Train data label inspection:', label_batch.numpy())
    print(image_batch.numpy().shape, len(image_batch.numpy()), label_batch.numpy().shape)
    validation_df = kaggle_image_augmentation(validation_df, strategy) # No data augmentation for Validation dataset
    # Inspect Validation dataset
    for image_batch, label_batch in validation_df.take(1):
        pass
    print('Main: Validation data label inspection:', label_batch.numpy())
    print(image_batch.numpy().shape, len(image_batch.numpy()), label_batch.numpy().shape)
    copy_validation_df = validation_df # Keep a copy of the original validation_df for model evaluation
    test_df = test_df.batch(DL_BATCH_SIZE * strategy.num_replicas_in_sync) # Prepare batches for DL
    test_df = test_df.prefetch(tf.data.experimental.AUTOTUNE) # The next batch will always be ready for processing for the next iteration
    # Inspect test dataset
    for image_batch in test_df.take(1):
        pass
    print('Main: Test inspection:')
    print('type(image_batch) ==> ', type(image_batch))
    print('len(image_batch) ==> ', len(image_batch))
    #print(image_batch)
    print('Main: Dataset: %d training images, %d validation images, %d unlabeled test images' % (train_df_size, validation_df_size, test_df_size))
    # 2. Do a basic DL evaluation as reference for the end
    print('----------------------------- kaggle_dl_quick_and_dirty -----------------------------')
    # FIXME Enhance function kaggle_dl_quick_and_dirty to support CNN and more
    model = None
    if OUTPUT_IS_REGRESSION: # Use regression algorithms
        with strategy.scope():
            pass # TODO
            # End of 'with' statement
    else:
        with strategy.scope():
            model = kaggle_create_sequential_classifier_model(
                                                              #strategy
                                                              #p_input_shape = DL_INPUT_SHAPE, 
                                                              #p_drop_rate = DL_DROP_RATE,
                                                              p_loss = 'categorical_crossentropy', 
                                                              p_metrics = ['categorical_accuracy'],
                                                              p_pretrained_layers = ['VGG19'], 
                                                              p_image_size = IMAGE_SIZE, 
                                                              p_class_num = len(IMAGE_CLASSES)
                                                              )
    with strategy.scope():
        loss, accuracy = model.evaluate(validation_df, steps = 16, verbose = 2)
        print('Loss: %.2f' % loss)
        print('Accuracy: %.2f' % accuracy)
        # End of 'with' statement

    raise Exception('Stop')

    if OUTPUT_IS_REGRESSION: # Use regression algorithms
        with strategy.scope():
            pass # TODO
        # End of 'with' statement
    else:
        # Using kaggle_check_models() and KerasClassifier require too many processing time, just use standard fit() method
        with strategy.scope():
            models.append(('VGG19', kaggle_create_sequential_classifier_model(
                                                                              #strategy
                                                                              #p_input_shape = DL_INPUT_SHAPE, 
                                                                              #p_drop_rate = DL_DROP_RATE,
                                                                              p_loss = 'categorical_crossentropy', 
                                                                              p_metrics = ['categorical_accuracy'],
                                                                              p_pretrained_layers = ['VGG19'], 
                                                                              p_image_size = IMAGE_SIZE, 
                                                                              p_class_num = len(IMAGE_CLASSES)
                                                                              )))
            models.append(('DenseNet201', kaggle_create_sequential_classifier_model(
                                                                                 #strategy
                                                                                 #p_input_shape = DL_INPUT_SHAPE, 
                                                                                 #p_drop_rate = DL_DROP_RATE,
                                                                                 p_loss = 'categorical_crossentropy', 
                                                                                 p_metrics = ['categorical_accuracy'],
                                                                                 p_pretrained_layers = ['DenseNet201'], 
                                                                                 p_image_size = IMAGE_SIZE, 
                                                                                 p_class_num = len(IMAGE_CLASSES)
                                                                                 )))
    with strategy.scope():
        print('----------------------------- kaggle_check_dl_models -----------------------------')
        results = []
        histories = []
        trained_models = []
        for name, model in models:
            print('kaggle_check_dl_models: Processing %s with type %s' % (name, type(model)))
            result, history, trained_model = kaggle_check_dl_model(strategy, model, train_df, train_df_size, validation_df, validation_df_size)
            results.append(result)
            histories.append(history)
            trained_models.append(trained_model)
            print('kaggle_check_dl_models: result: ', result)
            #print('kaggle_check_dl_models: history: ', history)
            #break # For debug
            # End of 'for' statement
        ml = trained_models[0]
        # TODO Add support of regressor!
    # End of 'with' statement

    print('Main: End of processing')
    # End of function kaggle_main

Entry point

In [ ]:
# Entry point
print("Starting at ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
kaggle_main()
print("Ending at ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

***If you liked this Notebook, please upvote.
Gives Motivation to make new Notebooks :)***